# Increase Predictions by 50% with Smart Prompting

![Author](https://img.shields.io/badge/Author-Nhi%20Yen-brightgreen)
[![Medium](https://img.shields.io/badge/Medium-Follow%20Me-blue)](https://medium.com/@yennhi95zz/subscribe)
[![GitHub](https://img.shields.io/badge/GitHub-Follow%20Me-lightgrey)](https://github.com/yennhi95zz)
[![Kaggle](https://img.shields.io/badge/Kaggle-Follow%20Me-orange)](https://www.kaggle.com/nhiyen/code)
[![LinkedIn](https://img.shields.io/badge/LinkedIn-Connect%20with%20Me-informational)](https://www.linkedin.com/in/yennhi95zz/)
[![Upwork](https://img.shields.io/badge/Upwork-Hire%20Me-brightgreen)](https://www.upwork.com/freelancers/~018cb35a4fd005fbff)

This notebook is associated with the articles/ project below:

- Find the full code on this [GitHub repository](https://github.com/yennhi95zz/llm-prompt-engineering).
- Explore a detailed explanation in my [Medium article](https://medium.com/@yennhi95zz/increase-predictions-by-50-with-smart-prompting-981385fb10af).

👉Get UNLIMITED access to every story on Medium with just $1/week ▶ [HERE](https://medium.com/@yennhi95zz/membership)

👉Troubling with LLM? Hire me in [Upwork](https://medium.com/r/?url=https%3A%2F%2Fwww.upwork.com%2Ffreelancers%2F~018cb35a4fd005fbff).

In [3]:
import os
from data import messages
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

In [4]:
MY_OPENAI_KEY = os.getenv("MY_OPENAI_KEY")

# Define the Initial Prompt

In [5]:
template = """Is the sentence in the backticks below about anger? Answer only "yes" or "no".
            ```{sentence}```
        """

prompt = PromptTemplate.from_template(template)

In [6]:
llm = OpenAI(openai_api_key=MY_OPENAI_KEY,
             model="gpt-3.5-turbo-instruct",
             temperature=0)
llm_chain = prompt | llm

# Define the predict() function

In [7]:
def predict(prompt):
    # Initialize lists to store true labels, predicted labels, and responses
    y_true = []
    y_pred = []
    responses = []

    # Iterate over each message in the messages dataset
    for message in messages:
        # Get the response from the model using the provided llm_chain
        response = llm_chain.invoke(prompt.format(sentence=message['text']))

        # Append the true label and the predicted label to their respective lists
        y_true.append(message['label'])
        y_pred.append(1 if 'yes' in response.lower() else 0)

        # Append the response to the responses list
        responses.append(response)

    # Initialize a 2x2 confusion matrix
    cm = [[0, 0], [0, 0]]

    # Populate the confusion matrix based on true and predicted labels
    for true_label, pred_label in zip(y_true, y_pred):
        cm[true_label][pred_label] += 1

    # Print the confusion matrix with descriptive message
    print("Confusion Matrix:")
    print("[[True Negative, False Positive], [False Negative, True Positive]]")
    for row in cm:
        print(row)

    # Calculate precision, recall, F-score, and accuracy
    precision = cm[1][1] / (cm[1][1] + cm[0][1]) if (cm[1][1] + cm[0][1]) != 0 else 0
    recall = cm[1][1] / (cm[1][1] + cm[1][0]) if (cm[1][1] + cm[1][0]) != 0 else 0
    f_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
    accuracy = (cm[0][0] + cm[1][1]) / sum(map(sum, cm)) if sum(map(sum, cm)) != 0 else 0

    # Print evaluation metrics with descriptive messages
    print(f"\nPrecision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F-score: {f_score:.2f}")
    print(f"Accuracy: {accuracy:.2f}")

    # Compile results into a dictionary
    result = { 'f_score': f_score, 'accuracy': accuracy, 'truth': y_true, 'predicted': y_pred, 'responses': responses }

    # Print a summary of misclassified messages with descriptive messages
    print("\nSummary of Misclassified Messages:")
    for i in range(len(messages)):
        if result['predicted'][i] != result['truth'][i]:
            print(f"Message: {messages[i]['text']}\nTruth: {result['truth'][i]}, Predicted: {result['predicted'][i]}, Response: {result['responses'][i]}\n")

    # return result


# Evaluate the initial prompt

In [8]:
predict(prompt)

Confusion Matrix:
[[True Negative, False Positive], [False Negative, True Positive]]
[34, 0]
[17, 4]

Precision: 1.00
Recall: 0.19
F-score: 0.32
Accuracy: 0.69

Summary of Misclassified Messages:
Message: I hate this food.
Truth: 1, Predicted: 0, Response: 
no

Message: This movie is terrible.
Truth: 1, Predicted: 0, Response: 
no

Message: I can't believe you would do something like that!
Truth: 1, Predicted: 0, Response: 
no

Message: This is absolutely unacceptable!
Truth: 1, Predicted: 0, Response: 
no

Message: How could you be so thoughtless?
Truth: 1, Predicted: 0, Response: 
no

Message: You have no idea how much you've hurt me.
Truth: 1, Predicted: 0, Response: 
No

Message: You've really crossed the line this time.
Truth: 1, Predicted: 0, Response: 
No

Message: I won't stand for this kind of behavior.
Truth: 1, Predicted: 0, Response: 
no

Message: Tied to charger for conversations lasting more than 45 minutes.MAJOR PROBLEMS!!
Truth: 1, Predicted: 0, Response: 
No

Message: 

# Improvement 1 -  Add a role

In [9]:
template = """
            You are a sentiment bot. 
            Is the sentence in the backticks below about anger:
            ```{sentence}```
            Your answer is only "yes" or "no".
        """

prompt = PromptTemplate.from_template(template)

predict(prompt)

Confusion Matrix:
[[True Negative, False Positive], [False Negative, True Positive]]
[34, 0]
[7, 14]

Precision: 1.00
Recall: 0.67
F-score: 0.80
Accuracy: 0.87

Summary of Misclassified Messages:
Message: I won't stand for this kind of behavior.
Truth: 1, Predicted: 0, Response: 
No

Message: Tied to charger for conversations lasting more than 45 minutes.MAJOR PROBLEMS!!
Truth: 1, Predicted: 0, Response: 
No

Message: Needless to say I wasted my money.
Truth: 1, Predicted: 0, Response: 
No

Message: The commercials are the most misleading.
Truth: 1, Predicted: 0, Response: 
no

Message: I have yet to run this new battery below two bars and thats three days without charging. I bought it for my mother and she had a problem with the battery.
Truth: 1, Predicted: 0, Response: 
No

Message: This is a simple little phone to use but the breakage is unacceptible.
Truth: 1, Predicted: 0, Response: 
no

Message: It is unusable in a moving car at freeway speed.
Truth: 1, Predicted: 0, Response: 


# Improvement 2 - Define a task

In [10]:
template = """
            You are a sentiment bot. This is a classification task.
            Your job is to determine if the sentence in the backticks below is about anger:
            ```{sentence}```
            Your answer is only "yes" or "no".
        """

prompt = PromptTemplate.from_template(template)

predict(prompt)

Confusion Matrix:
[[True Negative, False Positive], [False Negative, True Positive]]
[34, 0]
[8, 13]

Precision: 1.00
Recall: 0.62
F-score: 0.76
Accuracy: 0.85

Summary of Misclassified Messages:
Message: I won't stand for this kind of behavior.
Truth: 1, Predicted: 0, Response: 
No

Message: Tied to charger for conversations lasting more than 45 minutes.MAJOR PROBLEMS!!
Truth: 1, Predicted: 0, Response: 
No

Message: Needless to say I wasted my money.
Truth: 1, Predicted: 0, Response: 
No

Message: I advise EVERYONE DO NOT BE FOOLED!
Truth: 1, Predicted: 0, Response: 
No

Message: The commercials are the most misleading.
Truth: 1, Predicted: 0, Response: 
No

Message: I have yet to run this new battery below two bars and thats three days without charging. I bought it for my mother and she had a problem with the battery.
Truth: 1, Predicted: 0, Response: 
No

Message: This is a simple little phone to use but the breakage is unacceptible.
Truth: 1, Predicted: 0, Response: 
No

Message: 

# Improvement 3 — Describe the context

In [11]:
template = """
            You are a sentiment bot. This is a classification task.
            Given a sentence about customer feedback.
            Your job is to determine if the sentence in the backticks below is about anger:
            ```{sentence}```
            Your answer is only "yes" or "no".
        """

prompt = PromptTemplate.from_template(template)

predict(prompt)


Confusion Matrix:
[[True Negative, False Positive], [False Negative, True Positive]]
[34, 0]
[5, 16]

Precision: 1.00
Recall: 0.76
F-score: 0.86
Accuracy: 0.91

Summary of Misclassified Messages:
Message: Tied to charger for conversations lasting more than 45 minutes.MAJOR PROBLEMS!!
Truth: 1, Predicted: 0, Response: 
No

Message: I advise EVERYONE DO NOT BE FOOLED!
Truth: 1, Predicted: 0, Response: 
No

Message: The commercials are the most misleading.
Truth: 1, Predicted: 0, Response: 
No

Message: I have yet to run this new battery below two bars and thats three days without charging. I bought it for my mother and she had a problem with the battery.
Truth: 1, Predicted: 0, Response: 
No

Message: It is unusable in a moving car at freeway speed.
Truth: 1, Predicted: 0, Response: 
No



# Improvement 4 - Provide a definition

In [12]:
template = """
            You are a sentiment bot. This is a classification task.
            Given a sentence about customer feedback.
            Your job is to determine if the sentence in the backticks below is about anger:
            ```{sentence}```
            This is difinition of "anger":
            - Anger is an intense emotional state of displeasure with someone or something.
            - Anger is an emotion characterized by antagonism toward someone or something you feel has deliberately done you wrong.
            - Anger is one of the basic human emotions, as elemental as happiness, sadness, anxiety, or disgust.
            Your answer is only "yes" or "no".
        """

prompt = PromptTemplate.from_template(template)

predict(prompt)

Confusion Matrix:
[[True Negative, False Positive], [False Negative, True Positive]]
[34, 0]
[3, 18]

Precision: 1.00
Recall: 0.86
F-score: 0.92
Accuracy: 0.95

Summary of Misclassified Messages:
Message: Tied to charger for conversations lasting more than 45 minutes.MAJOR PROBLEMS!!
Truth: 1, Predicted: 0, Response: 
No

Message: I have yet to run this new battery below two bars and thats three days without charging. I bought it for my mother and she had a problem with the battery.
Truth: 1, Predicted: 0, Response: 
No

Message: It is unusable in a moving car at freeway speed.
Truth: 1, Predicted: 0, Response: 
No



# Improvement 5 - Provide more guidance and examples

In [13]:
template = """
            You are a sentiment bot. This is a classification task.
            Given a sentence about customer feedback.
            Your job is to determine if the sentence in the backticks below is about anger:
            ```{sentence}```
            This is difinition of "anger":
            - Anger is an intense emotional state of displeasure with someone or something.
            - Anger is an emotion characterized by antagonism toward someone or something you feel has deliberately done you wrong.
            - Anger is one of the basic human emotions, as elemental as happiness, sadness, anxiety, or disgust.
            To define a sentence that expresses "anger," look for features such as:
            - Emphatic Language: Use of strong, forceful words, often with CAPITALIZATION or exclamation marks to convey intensity. Example: "I can't believe you did that!"; "I am FURIOUS!"
            - Negative Connotations: Words that imply frustration, betrayal, or disapproval. Example: "This is completely unacceptable."
            - Directive or Accusatory Tone: Commands, accusations, or strong advice aimed at someone, indicating displeasure. Example: "You need to stop lying!"
            - Emotional Intensity: Expressions that reflect high emotional arousal, often indicating a loss of patience or tolerance. Example: "I've had enough of this nonsense!"
            - Context of Disappointment or Betrayal: Indicating that expectations have been unmet, leading to frustration. Example: "I trusted you, and this is how you repay me?"; "You assured me this would be taken care of, but once again, I find myself dealing with the same problem!"
            Other sample sentences to express anger: "I don'\t believe it!"; "Is it possible?"; "I could really do without it."; "It really gets on my nerves."
            Your answer is only "yes" or "no".
        """

prompt = PromptTemplate.from_template(template)

predict(prompt)

Confusion Matrix:
[[True Negative, False Positive], [False Negative, True Positive]]
[32, 2]
[1, 20]

Precision: 0.91
Recall: 0.95
F-score: 0.93
Accuracy: 0.95

Summary of Misclassified Messages:
Message: I went on Motorolas website and followed all directions but could not get it to pair again.
Truth: 0, Predicted: 1, Response: 
Yes

Message: I have yet to run this new battery below two bars and thats three days without charging. I bought it for my mother and she had a problem with the battery.
Truth: 1, Predicted: 0, Response: 
No

Message: Doesnt hold charge.
Truth: 0, Predicted: 1, Response: 
Yes



# Add more explanation

In [14]:
template = """
            You are a sentiment bot. This is a classification task.
            Given a sentence about customer feedback.
            Your job is to determine if the sentence in the backticks below is about anger:
            ```{sentence}```
            This is difinition of "anger":
            - Anger is an intense emotional state of displeasure with someone or something.
            - Anger is an emotion characterized by antagonism toward someone or something you feel has deliberately done you wrong.
            - Anger is one of the basic human emotions, as elemental as happiness, sadness, anxiety, or disgust.
            To define a sentence that expresses "anger," look for features such as:
            - Emphatic Language: Use of strong, forceful words, often with CAPITALIZATION or exclamation marks to convey intensity. Example: "I can't believe you did that!"; "I am FURIOUS!"
            - Negative Connotations: Words that imply frustration, betrayal, or disapproval. Example: "This is completely unacceptable."
            - Directive or Accusatory Tone: Commands, accusations, or strong advice aimed at someone, indicating displeasure. Example: "You need to stop lying!"
            - Emotional Intensity: Expressions that reflect high emotional arousal, often indicating a loss of patience or tolerance. Example: "I've had enough of this nonsense!"
            - Context of Disappointment or Betrayal: Indicating that expectations have been unmet, leading to frustration. Example: "I trusted you, and this is how you repay me?"; "You assured me this would be taken care of, but once again, I find myself dealing with the same problem!"
            There are 3 types of Anger:
            - Aggressive Anger: "Shut Up!"; "Get Lost"; "You'll be sorry"
            - Passive: "I\'m sorry to have to tell you this…"; "Please don\'t get angry or upset with me";
            - Assertive: "I feel"; "I think"; "I am angry about…"; "I would like you to…"; "I don't understand"
            Your answer is only "yes" or "no".
        """

prompt = PromptTemplate.from_template(template)

predict(prompt)


Confusion Matrix:
[[True Negative, False Positive], [False Negative, True Positive]]
[32, 2]
[1, 20]

Precision: 0.91
Recall: 0.95
F-score: 0.93
Accuracy: 0.95

Summary of Misclassified Messages:
Message: I went on Motorolas website and followed all directions but could not get it to pair again.
Truth: 0, Predicted: 1, Response: 
Yes

Message: I have yet to run this new battery below two bars and thats three days without charging. I bought it for my mother and she had a problem with the battery.
Truth: 1, Predicted: 0, Response: 
No

Message: Doesnt hold charge.
Truth: 0, Predicted: 1, Response: 
Yes



# More examples

In [15]:
# "I don’t believe it!"; "Is it possible?"; "I could really do without it."; "It really gets on my nerves."
template = """
            You are a sentiment bot. This is a classification task.
            Given a sentence about customer feedback.
            Your job is to determine if the sentence in the backticks below is about anger:
            ```{sentence}```
            This is difinition of "anger":
            - Anger is an intense emotional state of displeasure with someone or something.
            - Anger is an emotion characterized by antagonism toward someone or something you feel has deliberately done you wrong.
            - Anger is one of the basic human emotions, as elemental as happiness, sadness, anxiety, or disgust.
            To define a sentence that expresses "anger," look for features such as:
            - Emphatic Language: Use of strong, forceful words, often with CAPITALIZATION or exclamation marks to convey intensity. Example: "I can't believe you did that!"; "I am FURIOUS!"
            - Negative Connotations: Words that imply frustration, betrayal, or disapproval. Example: "This is completely unacceptable."
            - Directive or Accusatory Tone: Commands, accusations, or strong advice aimed at someone, indicating displeasure. Example: "You need to stop lying!"
            - Emotional Intensity: Expressions that reflect high emotional arousal, often indicating a loss of patience or tolerance. Example: "I've had enough of this nonsense!"
            - Context of Disappointment or Betrayal: Indicating that expectations have been unmet, leading to frustration. Example: "I trusted you, and this is how you repay me?"; "You assured me this would be taken care of, but once again, I find myself dealing with the same problem!"
            There are 3 types of Anger:
            - Aggressive Anger: "Shut Up!"; "Get Lost"; "You'll be sorry"
            - Passive: "I\'m sorry to have to tell you this…"; "Please don\'t get angry or upset with me";
            - Assertive: "I feel"; "I think"; "I am angry about…"; "I would like you to…"; "I don't understand"
            Other sample sentences to express anger: "I don\'t believe it!"; "Is it possible?"; "I could really do without it."; "It really gets on my nerves."
            Your answer is only "yes" or "no".
        """

prompt = PromptTemplate.from_template(template)

predict(prompt)

Confusion Matrix:
[[True Negative, False Positive], [False Negative, True Positive]]
[31, 3]
[1, 20]

Precision: 0.87
Recall: 0.95
F-score: 0.91
Accuracy: 0.93

Summary of Misclassified Messages:
Message: If you are Razr owner...you must have this!
Truth: 0, Predicted: 1, Response: 
Yes

Message: I went on Motorolas website and followed all directions but could not get it to pair again.
Truth: 0, Predicted: 1, Response: 
Yes

Message: I have yet to run this new battery below two bars and thats three days without charging. I bought it for my mother and she had a problem with the battery.
Truth: 1, Predicted: 0, Response: 
No

Message: Doesnt hold charge.
Truth: 0, Predicted: 1, Response: 
yes

